# Working with Window Functions in PySPark

Window Functions helps us to compare current row with other rows in the same dataframe, calculating running totals , sequencing of events and sessionization of transactions etc. This notebook is compiled from the following blog posts:

+ https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html 
+ http://alvinhenrick.com/2017/05/16/apache-spark-analytical-window-functions/ 
+ https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-functions-windows.html 

In [1]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [2]:
# setting random seed for notebook reproducability
rnd_seed=23
np.random.seed=23
np.random.set_state=23

In [3]:
os.environ['SPARK_HOME']

'/Users/anindyas/work/spark-2.2.0-bin-hadoop2.6'

In [4]:
spark = SparkSession.builder.master("local[2]").appName("retail_database_analysis").getOrCreate()

In [5]:
spark

In [6]:
sc = spark.sparkContext
sc

<SparkContext master=local[2] appName=retail_database_analysis>

In [7]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

## Employee Data

In [8]:
# Create Sample Dataframe
employees = [
    (7369, "SMITH", "CLERK", 7902, "17-Dec-80", 800, 20, 10),
    (7499, "ALLEN", "SALESMAN", 7698, "20-Feb-81", 1600, 300, 30),
    (7521, "WARD", "SALESMAN", 7698, "22-Feb-81", 1250, 500, 30),
    (7566, "JONES", "MANAGER", 7839, "2-Apr-81", 2975, 0, 20),
    (7654, "MARTIN", "SALESMAN", 7698, "28-Sep-81", 1250, 1400, 30),
    (7698, "BLAKE", "MANAGER", 7839, "1-May-81", 2850, 0, 30),
    (7782, "CLARK", "MANAGER", 7839, "9-Jun-81", 2450, 0, 10),
    (7788, "SCOTT", "ANALYST", 7566, "19-Apr-87", 3000, 0, 20),
    (7629, "ALEX", "SALESMAN", 7698, "28-Sep-79", 1150, 1400, 30),
    (7839, "KING", "PRESIDENT", 0, "17-Nov-81", 5000, 0, 10),
    (7844, "TURNER", "SALESMAN", 7698, "8-Sep-81", 1500, 0, 30),
    (7876, "ADAMS", "CLERK", 7788, "23-May-87", 1100, 0, 20)    
]

In [9]:
emp_df = spark.createDataFrame(employees, ["empno", "ename", "job", "mgr", "hiredate", "sal", "comm", "deptno"])

In [10]:
emp_df.show(20)

+-----+------+---------+----+---------+----+----+------+
|empno| ename|      job| mgr| hiredate| sal|comm|deptno|
+-----+------+---------+----+---------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-Dec-80| 800|  20|    10|
| 7499| ALLEN| SALESMAN|7698|20-Feb-81|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-Feb-81|1250| 500|    30|
| 7566| JONES|  MANAGER|7839| 2-Apr-81|2975|   0|    20|
| 7654|MARTIN| SALESMAN|7698|28-Sep-81|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839| 1-May-81|2850|   0|    30|
| 7782| CLARK|  MANAGER|7839| 9-Jun-81|2450|   0|    10|
| 7788| SCOTT|  ANALYST|7566|19-Apr-87|3000|   0|    20|
| 7629|  ALEX| SALESMAN|7698|28-Sep-79|1150|1400|    30|
| 7839|  KING|PRESIDENT|   0|17-Nov-81|5000|   0|    10|
| 7844|TURNER| SALESMAN|7698| 8-Sep-81|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-May-87|1100|   0|    20|
+-----+------+---------+----+---------+----+----+------+



In [11]:
emp_df.printSchema()

root
 |-- empno: long (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: long (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: long (nullable = true)
 |-- comm: long (nullable = true)
 |-- deptno: long (nullable = true)



In [12]:
emp_df.createOrReplaceTempView("emp")

In [13]:
import re

def strip_margin(text):
    nomargin = re.sub('\n[ \t]*\|', ' ', text)
    trimmed = re.sub('\s+', ' ', nomargin)
    return trimmed

### Rank salary within each department:

**SQL Using rank() Window Function:**

In [14]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, RANK() OVER (partition by deptno ORDER BY sal desc) as rank
          |FROM emp
        """)).show(20)

+-----+------+---------+------+----+----+
|empno| ename|      job|deptno| sal|rank|
+-----+------+---------+------+----+----+
| 7839|  KING|PRESIDENT|    10|5000|   1|
| 7782| CLARK|  MANAGER|    10|2450|   2|
| 7369| SMITH|    CLERK|    10| 800|   3|
| 7698| BLAKE|  MANAGER|    30|2850|   1|
| 7499| ALLEN| SALESMAN|    30|1600|   2|
| 7844|TURNER| SALESMAN|    30|1500|   3|
| 7521|  WARD| SALESMAN|    30|1250|   4|
| 7654|MARTIN| SALESMAN|    30|1250|   4|
| 7629|  ALEX| SALESMAN|    30|1150|   6|
| 7788| SCOTT|  ANALYST|    20|3000|   1|
| 7566| JONES|  MANAGER|    20|2975|   2|
| 7876| ADAMS|    CLERK|    20|1100|   3|
+-----+------+---------+------+----+----+



**DF API Using Window Function:**

First of all we will need to define the window we will be working on i.e. we will partition by department (deptno) and order by salary (sal).

In [15]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [16]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.rank().over(windowSpec).alias('rank')).show(20)

+-----+------+---------+------+----+----+
|empno| ename|      job|deptno| sal|rank|
+-----+------+---------+------+----+----+
| 7839|  KING|PRESIDENT|    10|5000|   1|
| 7782| CLARK|  MANAGER|    10|2450|   2|
| 7369| SMITH|    CLERK|    10| 800|   3|
| 7698| BLAKE|  MANAGER|    30|2850|   1|
| 7499| ALLEN| SALESMAN|    30|1600|   2|
| 7844|TURNER| SALESMAN|    30|1500|   3|
| 7521|  WARD| SALESMAN|    30|1250|   4|
| 7654|MARTIN| SALESMAN|    30|1250|   4|
| 7629|  ALEX| SALESMAN|    30|1150|   6|
| 7788| SCOTT|  ANALYST|    20|3000|   1|
| 7566| JONES|  MANAGER|    20|2975|   2|
| 7876| ADAMS|    CLERK|    20|1100|   3|
+-----+------+---------+------+----+----+



### Dense Rank salary within each department:

**SQL Using dense_rank() Window Function:**

In [17]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, DENSE_RANK() OVER (partition by deptno ORDER BY sal desc) as dense_rank
          |FROM emp
        """)).show()

+-----+------+---------+------+----+----------+
|empno| ename|      job|deptno| sal|dense_rank|
+-----+------+---------+------+----+----------+
| 7839|  KING|PRESIDENT|    10|5000|         1|
| 7782| CLARK|  MANAGER|    10|2450|         2|
| 7369| SMITH|    CLERK|    10| 800|         3|
| 7698| BLAKE|  MANAGER|    30|2850|         1|
| 7499| ALLEN| SALESMAN|    30|1600|         2|
| 7844|TURNER| SALESMAN|    30|1500|         3|
| 7521|  WARD| SALESMAN|    30|1250|         4|
| 7654|MARTIN| SALESMAN|    30|1250|         4|
| 7629|  ALEX| SALESMAN|    30|1150|         5|
| 7788| SCOTT|  ANALYST|    20|3000|         1|
| 7566| JONES|  MANAGER|    20|2975|         2|
| 7876| ADAMS|    CLERK|    20|1100|         3|
+-----+------+---------+------+----+----------+



**DF API Using Window Function:**

In [18]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [19]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.dense_rank().over(windowSpec).alias('dense_rank')).show()

+-----+------+---------+------+----+----------+
|empno| ename|      job|deptno| sal|dense_rank|
+-----+------+---------+------+----+----------+
| 7839|  KING|PRESIDENT|    10|5000|         1|
| 7782| CLARK|  MANAGER|    10|2450|         2|
| 7369| SMITH|    CLERK|    10| 800|         3|
| 7698| BLAKE|  MANAGER|    30|2850|         1|
| 7499| ALLEN| SALESMAN|    30|1600|         2|
| 7844|TURNER| SALESMAN|    30|1500|         3|
| 7521|  WARD| SALESMAN|    30|1250|         4|
| 7654|MARTIN| SALESMAN|    30|1250|         4|
| 7629|  ALEX| SALESMAN|    30|1150|         5|
| 7788| SCOTT|  ANALYST|    20|3000|         1|
| 7566| JONES|  MANAGER|    20|2975|         2|
| 7876| ADAMS|    CLERK|    20|1100|         3|
+-----+------+---------+------+----+----------+



### Row Number within each department:

**SQL Using row_num() Window Function:**

In [20]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, ROW_NUMBER() OVER (partition by deptno ORDER BY sal desc) as row_num
          |FROM emp
        """)).show()

+-----+------+---------+------+----+-------+
|empno| ename|      job|deptno| sal|row_num|
+-----+------+---------+------+----+-------+
| 7839|  KING|PRESIDENT|    10|5000|      1|
| 7782| CLARK|  MANAGER|    10|2450|      2|
| 7369| SMITH|    CLERK|    10| 800|      3|
| 7698| BLAKE|  MANAGER|    30|2850|      1|
| 7499| ALLEN| SALESMAN|    30|1600|      2|
| 7844|TURNER| SALESMAN|    30|1500|      3|
| 7521|  WARD| SALESMAN|    30|1250|      4|
| 7654|MARTIN| SALESMAN|    30|1250|      5|
| 7629|  ALEX| SALESMAN|    30|1150|      6|
| 7788| SCOTT|  ANALYST|    20|3000|      1|
| 7566| JONES|  MANAGER|    20|2975|      2|
| 7876| ADAMS|    CLERK|    20|1100|      3|
+-----+------+---------+------+----+-------+



**DF API Using Window Function:**

In [21]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [22]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.row_number().over(windowSpec).alias('row_num')).show()

+-----+------+---------+------+----+-------+
|empno| ename|      job|deptno| sal|row_num|
+-----+------+---------+------+----+-------+
| 7839|  KING|PRESIDENT|    10|5000|      1|
| 7782| CLARK|  MANAGER|    10|2450|      2|
| 7369| SMITH|    CLERK|    10| 800|      3|
| 7698| BLAKE|  MANAGER|    30|2850|      1|
| 7499| ALLEN| SALESMAN|    30|1600|      2|
| 7844|TURNER| SALESMAN|    30|1500|      3|
| 7521|  WARD| SALESMAN|    30|1250|      4|
| 7654|MARTIN| SALESMAN|    30|1250|      5|
| 7629|  ALEX| SALESMAN|    30|1150|      6|
| 7788| SCOTT|  ANALYST|    20|3000|      1|
| 7566| JONES|  MANAGER|    20|2975|      2|
| 7876| ADAMS|    CLERK|    20|1100|      3|
+-----+------+---------+------+----+-------+



### Running Total (Salary) within each department:

**SQL Using sum() with Window Function:**

In [23]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, SUM(sal) OVER (partition by deptno ORDER BY sal desc) as running_total
          |FROM emp
        """)).show()

+-----+------+---------+------+----+-------------+
|empno| ename|      job|deptno| sal|running_total|
+-----+------+---------+------+----+-------------+
| 7839|  KING|PRESIDENT|    10|5000|         5000|
| 7782| CLARK|  MANAGER|    10|2450|         7450|
| 7369| SMITH|    CLERK|    10| 800|         8250|
| 7698| BLAKE|  MANAGER|    30|2850|         2850|
| 7499| ALLEN| SALESMAN|    30|1600|         4450|
| 7844|TURNER| SALESMAN|    30|1500|         5950|
| 7521|  WARD| SALESMAN|    30|1250|         8450|
| 7654|MARTIN| SALESMAN|    30|1250|         8450|
| 7629|  ALEX| SALESMAN|    30|1150|         9600|
| 7788| SCOTT|  ANALYST|    20|3000|         3000|
| 7566| JONES|  MANAGER|    20|2975|         5975|
| 7876| ADAMS|    CLERK|    20|1100|         7075|
+-----+------+---------+------+----+-------------+



**DF API Using sum() with Window Function:**

In [24]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [25]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.sum('sal').over(windowSpec).alias('running_total')).show()

+-----+------+---------+------+----+-------------+
|empno| ename|      job|deptno| sal|running_total|
+-----+------+---------+------+----+-------------+
| 7839|  KING|PRESIDENT|    10|5000|         5000|
| 7782| CLARK|  MANAGER|    10|2450|         7450|
| 7369| SMITH|    CLERK|    10| 800|         8250|
| 7698| BLAKE|  MANAGER|    30|2850|         2850|
| 7499| ALLEN| SALESMAN|    30|1600|         4450|
| 7844|TURNER| SALESMAN|    30|1500|         5950|
| 7521|  WARD| SALESMAN|    30|1250|         8450|
| 7654|MARTIN| SALESMAN|    30|1250|         8450|
| 7629|  ALEX| SALESMAN|    30|1150|         9600|
| 7788| SCOTT|  ANALYST|    20|3000|         3000|
| 7566| JONES|  MANAGER|    20|2975|         5975|
| 7876| ADAMS|    CLERK|    20|1100|         7075|
+-----+------+---------+------+----+-------------+



In [26]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [27]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.round(F.avg('sal').over(windowSpec), 2).alias('moving_avg')).show()

+-----+------+---------+------+----+----------+
|empno| ename|      job|deptno| sal|moving_avg|
+-----+------+---------+------+----+----------+
| 7839|  KING|PRESIDENT|    10|5000|    5000.0|
| 7782| CLARK|  MANAGER|    10|2450|    3725.0|
| 7369| SMITH|    CLERK|    10| 800|    2750.0|
| 7698| BLAKE|  MANAGER|    30|2850|    2850.0|
| 7499| ALLEN| SALESMAN|    30|1600|    2225.0|
| 7844|TURNER| SALESMAN|    30|1500|   1983.33|
| 7521|  WARD| SALESMAN|    30|1250|    1800.0|
| 7654|MARTIN| SALESMAN|    30|1250|    1690.0|
| 7629|  ALEX| SALESMAN|    30|1150|    1600.0|
| 7788| SCOTT|  ANALYST|    20|3000|    3000.0|
| 7566| JONES|  MANAGER|    20|2975|    2987.5|
| 7876| ADAMS|    CLERK|    20|1100|   2358.33|
+-----+------+---------+------+----+----------+



### Next Salary within each department:
Lead function allows us to compare current row with subsequent rows within each partition depending on the second argument (offset) which is by default set to 1 i.e. next row but we can change that parameter 2 to compare against every other row. The 3rd parameter is default value to be returned when no subsequent values exists or null.

**SQL Using lead() with Window Function:**

In [28]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, LEAD(sal, 1) OVER (partition by deptno ORDER BY sal desc) as next_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|next_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    2450|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|    null|
| 7698| BLAKE|  MANAGER|    30|2850|    1600|
| 7499| ALLEN| SALESMAN|    30|1600|    1500|
| 7844|TURNER| SALESMAN|    30|1500|    1250|
| 7521|  WARD| SALESMAN|    30|1250|    1250|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|    null|
| 7788| SCOTT|  ANALYST|    20|3000|    2975|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|    null|
+-----+------+---------+------+----+--------+



**DF API Using lead() with Window Function:**

In [29]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [30]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.lead('sal', count=1, default=0).over(windowSpec).alias('next_val')).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|next_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    2450|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|       0|
| 7698| BLAKE|  MANAGER|    30|2850|    1600|
| 7499| ALLEN| SALESMAN|    30|1600|    1500|
| 7844|TURNER| SALESMAN|    30|1500|    1250|
| 7521|  WARD| SALESMAN|    30|1250|    1250|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|       0|
| 7788| SCOTT|  ANALYST|    20|3000|    2975|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|       0|
+-----+------+---------+------+----+--------+



### Previous Salary within each department:
Lag function allows us to compare current row with preceding rows within each partition depending on the second argument (offset) which is by default set to 1 i.e. next row but we can change that parameter 2 to compare against every other row. The 3rd parameter is default value to be returned when no subsequent values exists or null.

**SQL Using lag() with Window Function:**

In [31]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, LAG(sal, 1) OVER (partition by deptno ORDER BY sal desc) as prev_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|prev_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    null|
| 7782| CLARK|  MANAGER|    10|2450|    5000|
| 7369| SMITH|    CLERK|    10| 800|    2450|
| 7698| BLAKE|  MANAGER|    30|2850|    null|
| 7499| ALLEN| SALESMAN|    30|1600|    2850|
| 7844|TURNER| SALESMAN|    30|1500|    1600|
| 7521|  WARD| SALESMAN|    30|1250|    1500|
| 7654|MARTIN| SALESMAN|    30|1250|    1250|
| 7629|  ALEX| SALESMAN|    30|1150|    1250|
| 7788| SCOTT|  ANALYST|    20|3000|    null|
| 7566| JONES|  MANAGER|    20|2975|    3000|
| 7876| ADAMS|    CLERK|    20|1100|    2975|
+-----+------+---------+------+----+--------+



**DF API Using lag() with Window Function:**

In [32]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [33]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.lag('sal', count=1, default=0).over(windowSpec).alias('prev_val')).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|prev_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|       0|
| 7782| CLARK|  MANAGER|    10|2450|    5000|
| 7369| SMITH|    CLERK|    10| 800|    2450|
| 7698| BLAKE|  MANAGER|    30|2850|       0|
| 7499| ALLEN| SALESMAN|    30|1600|    2850|
| 7844|TURNER| SALESMAN|    30|1500|    1600|
| 7521|  WARD| SALESMAN|    30|1250|    1500|
| 7654|MARTIN| SALESMAN|    30|1250|    1250|
| 7629|  ALEX| SALESMAN|    30|1150|    1250|
| 7788| SCOTT|  ANALYST|    20|3000|       0|
| 7566| JONES|  MANAGER|    20|2975|    3000|
| 7876| ADAMS|    CLERK|    20|1100|    2975|
+-----+------+---------+------+----+--------+



### First Salary within each department:
First value within each partition .i.e. highest salary (we are using order by descending) within each department can be compared against every member within each department.

**SQL Using first() with Window Function:**

In [34]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, FIRST_VALUE(sal) OVER (partition by deptno ORDER BY sal desc) as first_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+---------+
|empno| ename|      job|deptno| sal|first_val|
+-----+------+---------+------+----+---------+
| 7839|  KING|PRESIDENT|    10|5000|     5000|
| 7782| CLARK|  MANAGER|    10|2450|     5000|
| 7369| SMITH|    CLERK|    10| 800|     5000|
| 7698| BLAKE|  MANAGER|    30|2850|     2850|
| 7499| ALLEN| SALESMAN|    30|1600|     2850|
| 7844|TURNER| SALESMAN|    30|1500|     2850|
| 7521|  WARD| SALESMAN|    30|1250|     2850|
| 7654|MARTIN| SALESMAN|    30|1250|     2850|
| 7629|  ALEX| SALESMAN|    30|1150|     2850|
| 7788| SCOTT|  ANALYST|    20|3000|     3000|
| 7566| JONES|  MANAGER|    20|2975|     3000|
| 7876| ADAMS|    CLERK|    20|1100|     3000|
+-----+------+---------+------+----+---------+



**DF API Using first() with Window Function:**

In [35]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [36]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.first('sal').over(windowSpec).alias('first_val')).show()

+-----+------+---------+------+----+---------+
|empno| ename|      job|deptno| sal|first_val|
+-----+------+---------+------+----+---------+
| 7839|  KING|PRESIDENT|    10|5000|     5000|
| 7782| CLARK|  MANAGER|    10|2450|     5000|
| 7369| SMITH|    CLERK|    10| 800|     5000|
| 7698| BLAKE|  MANAGER|    30|2850|     2850|
| 7499| ALLEN| SALESMAN|    30|1600|     2850|
| 7844|TURNER| SALESMAN|    30|1500|     2850|
| 7521|  WARD| SALESMAN|    30|1250|     2850|
| 7654|MARTIN| SALESMAN|    30|1250|     2850|
| 7629|  ALEX| SALESMAN|    30|1150|     2850|
| 7788| SCOTT|  ANALYST|    20|3000|     3000|
| 7566| JONES|  MANAGER|    20|2975|     3000|
| 7876| ADAMS|    CLERK|    20|1100|     3000|
+-----+------+---------+------+----+---------+



### Last Salary within each department:
Last value within each partition .i.e. lowest salary (we are using order by descending) within each department can be compared against every member within each department.

**SQL Using last() with Window Function:**

In [37]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, LAST_VALUE(sal) OVER (partition by deptno ORDER BY sal desc) as last_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|last_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    5000|
| 7782| CLARK|  MANAGER|    10|2450|    2450|
| 7369| SMITH|    CLERK|    10| 800|     800|
| 7698| BLAKE|  MANAGER|    30|2850|    2850|
| 7499| ALLEN| SALESMAN|    30|1600|    1600|
| 7844|TURNER| SALESMAN|    30|1500|    1500|
| 7521|  WARD| SALESMAN|    30|1250|    1250|
| 7654|MARTIN| SALESMAN|    30|1250|    1250|
| 7629|  ALEX| SALESMAN|    30|1150|    1150|
| 7788| SCOTT|  ANALYST|    20|3000|    3000|
| 7566| JONES|  MANAGER|    20|2975|    2975|
| 7876| ADAMS|    CLERK|    20|1100|    1100|
+-----+------+---------+------+----+--------+



**Oops!** what happened here the `last_val` has the same value as in `sal` column but we were expecting the lowest salary within the department in the last_val column so for that we really need to understand how the window operates and works. There are two types of frames `ROW` and `RANGE`.The details are explained in this [posts](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) from databricks.

*This happens because **default window frame is range between unbounded preceding and current row**, so the `last_value()` never looks beyond current row unless we change the frame.*

Last value fixed by supplying the window frame for `last_val()` to operate on. We will be using start frame current row and end frame unbounded following to get the last value.

In [38]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, 
          |    LAST_VALUE(sal) OVER (partition by deptno ORDER BY sal desc ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) as last_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|last_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|     800|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|     800|
| 7698| BLAKE|  MANAGER|    30|2850|    1150|
| 7499| ALLEN| SALESMAN|    30|1600|    1150|
| 7844|TURNER| SALESMAN|    30|1500|    1150|
| 7521|  WARD| SALESMAN|    30|1250|    1150|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|    1150|
| 7788| SCOTT|  ANALYST|    20|3000|    1100|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|    1100|
+-----+------+---------+------+----+--------+



**DF API Using last() with Window Function:**

In [39]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc()).rowsBetween(Window.currentRow, Window.unboundedFollowing)

In [40]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.last('sal').over(windowSpec).alias('last_val')).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|last_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|     800|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|     800|
| 7698| BLAKE|  MANAGER|    30|2850|    1150|
| 7499| ALLEN| SALESMAN|    30|1600|    1150|
| 7844|TURNER| SALESMAN|    30|1500|    1150|
| 7521|  WARD| SALESMAN|    30|1250|    1150|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|    1150|
| 7788| SCOTT|  ANALYST|    20|3000|    1100|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|    1100|
+-----+------+---------+------+----+--------+



##  Product Sales Data

In [41]:
product_revenues = [
  ("Thin",       "cell phone", 6000),
  ("Normal",     "tablet",     1500),
  ("Mini",       "tablet",     5500),
  ("Ultra thin", "cell phone", 5000),
  ("Very thin",  "cell phone", 6000),
  ("Big",        "tablet",     2500),
  ("Bendable",   "cell phone", 3000),
  ("Foldable",   "cell phone", 3000),
  ("Pro",        "tablet",     4500),
  ("Pro2",       "tablet",     6500)
]

In [42]:
prod_rev_df = spark.createDataFrame(product_revenues, ["product", "category", "revenue"])

In [43]:
prod_rev_df.show(20)

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Thin|cell phone|   6000|
|    Normal|    tablet|   1500|
|      Mini|    tablet|   5500|
|Ultra thin|cell phone|   5000|
| Very thin|cell phone|   6000|
|       Big|    tablet|   2500|
|  Bendable|cell phone|   3000|
|  Foldable|cell phone|   3000|
|       Pro|    tablet|   4500|
|      Pro2|    tablet|   6500|
+----------+----------+-------+



In [44]:
prod_rev_df.printSchema()

root
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- revenue: long (nullable = true)



In [45]:
prod_rev_df.createOrReplaceTempView("prod_rev")

### What are the best-selling and the second best-selling products in every category?

**SQL: Using dense_rank() with Window Function:**

In [46]:
spark.sql(strip_margin(
        """SELECT product, category, revenue, dense_rank
          |FROM (
          |    SELECT product, category, revenue, 
          |    DENSE_RANK(revenue) OVER (partition by category ORDER BY revenue desc) as dense_rank
          |    FROM prod_rev ) a
          |WHERE dense_rank <= 2
        """)).show()

+----------+----------+-------+----------+
|   product|  category|revenue|dense_rank|
+----------+----------+-------+----------+
|      Pro2|    tablet|   6500|         1|
|      Mini|    tablet|   5500|         2|
|      Thin|cell phone|   6000|         1|
| Very thin|cell phone|   6000|         1|
|Ultra thin|cell phone|   5000|         2|
+----------+----------+-------+----------+



**DF API Using dense_rank() with Window Function:**

In [47]:
windowSpec = Window.partitionBy(col('category')).orderBy(col('revenue').desc())

In [48]:
(prod_rev_df
 .select('product', 'category', 'revenue', F.dense_rank().over(windowSpec).alias('dense_rank'))
 .where('dense_rank <= 2')
 .show())

+----------+----------+-------+----------+
|   product|  category|revenue|dense_rank|
+----------+----------+-------+----------+
|      Pro2|    tablet|   6500|         1|
|      Mini|    tablet|   5500|         2|
|      Thin|cell phone|   6000|         1|
| Very thin|cell phone|   6000|         1|
|Ultra thin|cell phone|   5000|         2|
+----------+----------+-------+----------+



### What is the difference between the revenue of each product and the revenue of the best selling product in the same category as that product?

**SQL: Using RANGE BETWEEN with Window Function:**

In [49]:
spark.sql(strip_margin(
        """SELECT product, category, revenue, 
          |    MAX(revenue) OVER (partition by category ORDER BY revenue desc RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as max_rev,
          |    MAX(revenue) OVER (partition by category ORDER BY revenue desc RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) - revenue as rev_diff 
          |FROM prod_rev
        """)).show()

+----------+----------+-------+-------+--------+
|   product|  category|revenue|max_rev|rev_diff|
+----------+----------+-------+-------+--------+
|      Pro2|    tablet|   6500|   6500|       0|
|      Mini|    tablet|   5500|   6500|    1000|
|       Pro|    tablet|   4500|   6500|    2000|
|       Big|    tablet|   2500|   6500|    4000|
|    Normal|    tablet|   1500|   6500|    5000|
|      Thin|cell phone|   6000|   6000|       0|
| Very thin|cell phone|   6000|   6000|       0|
|Ultra thin|cell phone|   5000|   6000|    1000|
|  Bendable|cell phone|   3000|   6000|    3000|
|  Foldable|cell phone|   3000|   6000|    3000|
+----------+----------+-------+-------+--------+



**DF API: Using rangeBetween() with Window Function:**

In [50]:
windowSpec = (Window
              .partitionBy(prod_rev_df['category'])
              .orderBy(prod_rev_df['revenue'].desc())
              .rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing))

In [51]:
(prod_rev_df
 .select('product', 'category', 'revenue', 
             F.max('revenue').over(windowSpec).alias('max_rev'), 
             (F.max('revenue').over(windowSpec) - col('revenue')).alias('rev_diff'))
 .show())

+----------+----------+-------+-------+--------+
|   product|  category|revenue|max_rev|rev_diff|
+----------+----------+-------+-------+--------+
|      Pro2|    tablet|   6500|   6500|       0|
|      Mini|    tablet|   5500|   6500|    1000|
|       Pro|    tablet|   4500|   6500|    2000|
|       Big|    tablet|   2500|   6500|    4000|
|    Normal|    tablet|   1500|   6500|    5000|
|      Thin|cell phone|   6000|   6000|       0|
| Very thin|cell phone|   6000|   6000|       0|
|Ultra thin|cell phone|   5000|   6000|    1000|
|  Bendable|cell phone|   3000|   6000|    3000|
|  Foldable|cell phone|   3000|   6000|    3000|
+----------+----------+-------+-------+--------+



## Customer Expense Data

In this example dataset, there are two customers who have spent different amounts of money each day.

In [52]:
expense = [
    ('Alice', '5/1/2016', 50),
    ('Alice', '5/2/2016', 45),
    ('Alice', '5/3/2016', 55),
    ('Alice', '5/4/2016', 47),
    ('Alice', '5/5/2016', 19),
    ('Bob', '5/1/2016', 25),
    ('Bob', '5/4/2016', 29),
    ('Bob', '5/9/2016', 27),
    ('Bob', '5/12/2016', 32),
    ('Bob', '5/13/2016', 10)
]

In [53]:
expense_df = spark.createDataFrame(expense, ["name", "date", "amount_spent"])

In [54]:
expense_df.show(20)

+-----+---------+------------+
| name|     date|amount_spent|
+-----+---------+------------+
|Alice| 5/1/2016|          50|
|Alice| 5/2/2016|          45|
|Alice| 5/3/2016|          55|
|Alice| 5/4/2016|          47|
|Alice| 5/5/2016|          19|
|  Bob| 5/1/2016|          25|
|  Bob| 5/4/2016|          29|
|  Bob| 5/9/2016|          27|
|  Bob|5/12/2016|          32|
|  Bob|5/13/2016|          10|
+-----+---------+------------+



In [55]:
expense_df.createOrReplaceTempView("expense")

### What is the 3 days moving average for each customer:

**SQL: Using ROWS BETWEEN with specific values with Window Function:**

In [56]:
spark.sql(strip_margin(
        """SELECT name, date, amount_spent, 
          |    ROUND(AVG(amount_spent) OVER (partition by name ORDER BY date ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING), 2) as moving_avg
          |FROM expense
          |ORDER BY name
        """)).show()

+-----+---------+------------+----------+
| name|     date|amount_spent|moving_avg|
+-----+---------+------------+----------+
|Alice| 5/3/2016|          55|      49.0|
|Alice| 5/1/2016|          50|      47.5|
|Alice| 5/4/2016|          47|     40.33|
|Alice| 5/2/2016|          45|      50.0|
|Alice| 5/5/2016|          19|      33.0|
|  Bob| 5/1/2016|          25|      28.5|
|  Bob|5/13/2016|          10|     23.67|
|  Bob|5/12/2016|          32|     22.33|
|  Bob| 5/4/2016|          29|      22.0|
|  Bob| 5/9/2016|          27|      28.0|
+-----+---------+------------+----------+



**DF API: Using ROWS BETWEEN with specific values with Window Function:**

In [57]:
windowSpec = (Window
              .partitionBy(expense_df['name'])
              .orderBy(expense_df['date'])
              .rowsBetween(-1, 1))

In [58]:
(expense_df
 .select('name', 'date', 'amount_spent', F.round(F.avg('amount_spent').over(windowSpec), 2).alias('moving_avg'))
 .orderBy('name')
 .show())

+-----+---------+------------+----------+
| name|     date|amount_spent|moving_avg|
+-----+---------+------------+----------+
|Alice| 5/1/2016|          50|      47.5|
|Alice| 5/2/2016|          45|      50.0|
|Alice| 5/3/2016|          55|      49.0|
|Alice| 5/4/2016|          47|     40.33|
|Alice| 5/5/2016|          19|      33.0|
|  Bob| 5/1/2016|          25|      28.5|
|  Bob|5/12/2016|          32|     22.33|
|  Bob|5/13/2016|          10|     23.67|
|  Bob| 5/4/2016|          29|      22.0|
|  Bob| 5/9/2016|          27|      28.0|
+-----+---------+------------+----------+



### What is the cumulative amount spent by each customer:

**SQL: Using RANGE BETWEEN with Window Function:**

In [59]:
spark.sql(strip_margin(
        """SELECT name, date, amount_spent, 
          |    ROUND(SUM(amount_spent) OVER (partition by name ORDER BY date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 2) as moving_avg
          |FROM expense
          |ORDER BY name
        """)).show()

+-----+---------+------------+----------+
| name|     date|amount_spent|moving_avg|
+-----+---------+------------+----------+
|Alice| 5/1/2016|          50|        50|
|Alice| 5/2/2016|          45|        95|
|Alice| 5/3/2016|          55|       150|
|Alice| 5/4/2016|          47|       197|
|Alice| 5/5/2016|          19|       216|
|  Bob| 5/1/2016|          25|        25|
|  Bob|5/12/2016|          32|        57|
|  Bob|5/13/2016|          10|        67|
|  Bob| 5/4/2016|          29|        96|
|  Bob| 5/9/2016|          27|       123|
+-----+---------+------------+----------+



**DF API: Using RANGE BETWEEN with Window Function:**

In [60]:
windowSpec = (Window
              .partitionBy(expense_df['name'])
              .orderBy(expense_df['date'])
              .rangeBetween(Window.unboundedPreceding, Window.currentRow))

In [61]:
(expense_df
 .select('name', 'date', 'amount_spent', F.round(F.sum('amount_spent').over(windowSpec), 2).alias('cum_spent'))
 .orderBy('name')
 .show())

+-----+---------+------------+---------+
| name|     date|amount_spent|cum_spent|
+-----+---------+------------+---------+
|Alice| 5/3/2016|          55|      150|
|Alice| 5/1/2016|          50|       50|
|Alice| 5/4/2016|          47|      197|
|Alice| 5/2/2016|          45|       95|
|Alice| 5/5/2016|          19|      216|
|  Bob| 5/1/2016|          25|       25|
|  Bob|5/13/2016|          10|       67|
|  Bob|5/12/2016|          32|       57|
|  Bob| 5/4/2016|          29|       96|
|  Bob| 5/9/2016|          27|      123|
+-----+---------+------------+---------+



## Tractor Sales Data

In [62]:
sales = [
    (1, 'Jan-03', 141),
    (2, 'Feb-03', 157),
    (3, 'Mar-03', 185),
    (4, 'Apr-03', 199),
    (5, 'May-03', 203),
    (6, 'Jun-03', 189),
    (7, 'Jul-03', 207),
    (8, 'Aug-03', 207),
    (9, 'Sep-03', 171),
    (10, 'Oct-03', 150),
    (11, 'Nov-03', 138),
    (12, 'Dec-03', 165),
    (13, 'Jan-04', 145),
    (14, 'Feb-04', 168),
    (15, 'Mar-04', 197),
    (16, 'Apr-04', 208),
    (17, 'May-04', 210),
    (18, 'Jun-04', 209),
    (19, 'Jul-04', 238),
    (20, 'Aug-04', 238)
]

In [63]:
sales_df = spark.createDataFrame(sales, ['tid', 'month_year', 'sales']).cache()

In [64]:
sales_df.printSchema()

root
 |-- tid: long (nullable = true)
 |-- month_year: string (nullable = true)
 |-- sales: long (nullable = true)



In [65]:
sales_df.createOrReplaceTempView("sales")

### Running Difference of Current Sale from the Previous Sale:

**SQL: Using lag() with Window Function:**

In [66]:
spark.sql(strip_margin(
        """SELECT month_year, sales, 
          |    LAG(sales, 1) OVER (ORDER BY tid) as sales_lag_1,
          |    LAG(sales, 2) OVER (ORDER BY tid) as sales_lag_2,
          |    LAG(sales, 3) OVER (ORDER BY tid) as sales_lag_3,
          |    sales - LAG(sales, 1) OVER (ORDER BY tid) as diff_with_last_sales
          |FROM sales 
        """)).show()

+----------+-----+-----------+-----------+-----------+--------------------+
|month_year|sales|sales_lag_1|sales_lag_2|sales_lag_3|diff_with_last_sales|
+----------+-----+-----------+-----------+-----------+--------------------+
|    Jan-03|  141|       null|       null|       null|                null|
|    Feb-03|  157|        141|       null|       null|                  16|
|    Mar-03|  185|        157|        141|       null|                  28|
|    Apr-03|  199|        185|        157|        141|                  14|
|    May-03|  203|        199|        185|        157|                   4|
|    Jun-03|  189|        203|        199|        185|                 -14|
|    Jul-03|  207|        189|        203|        199|                  18|
|    Aug-03|  207|        207|        189|        203|                   0|
|    Sep-03|  171|        207|        207|        189|                 -36|
|    Oct-03|  150|        171|        207|        207|                 -21|
|    Nov-03|

**DF API: Using RANGE BETWEEN with Window Function:**

In [67]:
windowSpec = Window.orderBy(sales_df['tid'])

In [68]:
(sales_df
 .select('month_year', 'sales', 
         F.lag('sales', count=1).over(windowSpec).alias('sales_lag_1'),
         F.lag('sales', count=2).over(windowSpec).alias('sales_lag_2'),
         F.lag('sales', count=3).over(windowSpec).alias('sales_lag_3'),
         (col('sales') - F.lag('sales', count=1).over(windowSpec)).alias('diff_with_last_sales'))
 .show())

+----------+-----+-----------+-----------+-----------+--------------------+
|month_year|sales|sales_lag_1|sales_lag_2|sales_lag_3|diff_with_last_sales|
+----------+-----+-----------+-----------+-----------+--------------------+
|    Jan-03|  141|       null|       null|       null|                null|
|    Feb-03|  157|        141|       null|       null|                  16|
|    Mar-03|  185|        157|        141|       null|                  28|
|    Apr-03|  199|        185|        157|        141|                  14|
|    May-03|  203|        199|        185|        157|                   4|
|    Jun-03|  189|        203|        199|        185|                 -14|
|    Jul-03|  207|        189|        203|        199|                  18|
|    Aug-03|  207|        207|        189|        203|                   0|
|    Sep-03|  171|        207|        207|        189|                 -36|
|    Oct-03|  150|        171|        207|        207|                 -21|
|    Nov-03|

In [69]:
spark.stop()